In [1]:
from graphviz import Digraph

def parse_sequence_to_tree(sequence):
    dot = Digraph()
    dict_ = {'(':')', '{':'}', '[':']', '<':'>'}
    colors = ["blue", "red", "green", "yellow"]
    # dot.graph_attr['nodesep'] = '0.6'
    dot.edge_attr['penwidth'] = '0.6'
    dot.graph_attr['splines'] = 'line'

    # chars: .([{})]
    chars = set(sequence)
    degree = int(len(chars)/2) if '.' not in chars else int((len(chars) - 1)/2)


    # brackets: ([{}])
    brackets = chars
    if "." in brackets:
        brackets.remove(".")
    # ([{
    left_brackets = [b for b in brackets if b in "({[<"]

    # add basic Nodes and basic edge
    dot.node("R", label='', xlabel="root",width='0.1', height='0.1', shape='circle')
    for i in range(len(sequence)+2):
      dot.node(f"{i}", label='',xlabel=f"{i}",width='0.1', height='0.1', shape='circle')


    edges = []
    dot.edge("R", "0", arrowhead='none')
    edges.append(("R", "0"))




    for d in range(degree):

      split_seq = [
          c if (c == left_brackets[d] or c == dict_[left_brackets[d]]) else '.'
          for c in sequence
      ]
      # add node for bracket pairs
      num_pair = split_seq.count(left_brackets[d])
      for p in range(num_pair):
        dot.node(f"{d+1}-{p+1}", "", fillcolor=colors[d], style="filled",width='0.1', height='0.1', shape='circle')

      # add edges
      parent = "R"
      pair_count = 0
      pair_count_stack = []

      for c_ind in range(len(split_seq)):
        if split_seq[c_ind] == "." and ((parent, str(c_ind+1)) not in edges):
          dot.edge(parent, str(c_ind+1), arrowhead='none')
          edges.append((parent, str(c_ind+1)))
        if split_seq[c_ind] in "({[<":
          pair_count += 1
          new_parent = f"{d+1}-{pair_count}"
          dot.edge(parent, new_parent, arrowhead='none')
          parent = new_parent
          dot.edge(parent, str(c_ind+1), arrowhead='none')
          edges.append((parent, str(c_ind+1)))
          pair_count_stack.append(pair_count)
        if split_seq[c_ind] in ")}]>":
          pair_count_stack.pop()
          if len(pair_count_stack) == 0:
            new_parent = "R"
          else:
            new_parent = f"{d+1}-{pair_count_stack[-1]}"
          dot.edge(parent, str(c_ind+1), arrowhead='none')
          edges.append((parent, str(c_ind+1)))
          parent = new_parent



    dot.edge("R", str(len(sequence)+1), arrowhead='none')


    return dot


sequence = ".(([.{})).]()"
# separate1 = ".((....))..()"
# separate2 = "...[......].."
# separate3 = ".....{}......"
# sequence1 = "((((((([.((((][..{<..))))((((((.......))))))...]((((((}>.)()))))))))))))...."
# sequence2 = "((((((([.((((][..{<..))))((((((.......))))))...]((((((}>.)..))))))))))))...."
dot = parse_sequence_to_tree(sequence)
dot.render("sequence", format="png", cleanup=True)
dot.view()


'sequence.pdf'